In [20]:
from pathlib import Path
import numpy as np
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
from weight_diffusion.data.modelzoo_dataset import get_all_directories_for_a_path, parse_progress_csv

In [17]:
tokenizer = BertTokenizer.from_pretrained('tbs17/MathBERT', output_hidden_states=True)

In [18]:
data_dir = Path("../../data/tune_zoo_mnist_uniform")

In [22]:
prompt_latents_old = []
prompt_latents_new = []

for model_dir in tqdm(get_all_directories_for_a_path(data_dir)):
    model_dir_path = data_dir / model_dir
    checkpoint_dirs = get_all_directories_for_a_path(model_dir_path)
    model_progress_dict = parse_progress_csv(
        path_to_progress_csv= model_dir_path / "progress.csv"
    )
    for checkpoint_directory in checkpoint_dirs:
        checkpoint_key = int(checkpoint_directory[-6:])
        if checkpoint_key in model_progress_dict.keys():
            checkpoint_progress = model_progress_dict[checkpoint_key]
            prompt = (
                    f"The training loss is {checkpoint_progress['train_loss']:.4g}. "
                    f"The training accuracy is {checkpoint_progress['train_acc']:.4g}. "
                    f"The validation loss is {checkpoint_progress['validation_loss']:.4g}. "
                    f"The validation accuracy is {checkpoint_progress['validation_acc']:.4g}. "
                    f"The test loss is {checkpoint_progress['test_loss']:.4g}. "
                    f"The test accuracy is {checkpoint_progress['test_acc']:.4g}. "
                )
            prompt_latents_old += [tokenizer(
                prompt,
                max_length=77,
                return_tensors="pt",
                padding="max_length",
            )["input_ids"]]
            prompt = (
                f"{checkpoint_progress['train_loss']:.4g}. "
                f"{checkpoint_progress['train_acc']:.4g}. "
                f"{checkpoint_progress['validation_loss']:.4g}. "
                f"{checkpoint_progress['validation_acc']:.4g}. "
                f"{checkpoint_progress['test_loss']:.4g}. "
                f"{checkpoint_progress['test_acc']:.4g}. "
            )
            prompt_latents_new += [tokenizer(
                prompt,
                max_length=77,
                return_tensors="pt",
                padding="max_length",
            )["input_ids"]]

        else:
            continue

100%|██████████| 1000/1000 [01:22<00:00, 12.14it/s]


In [36]:
np.sum(np.std([x.numpy() for x in prompt_latents_old], axis=0))

294590.85068318126

In [37]:
np.sum(np.std([x.numpy() for x in prompt_latents_new], axis=0))

184859.41347257388